In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install -q -U "tensorflow-text==2.8.*"
!pip install -q tf-models-official==2.7.0
!pip install transformers
#!pip install focal_loss
#!pip install focal-loss

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import sklearn

#from focal_loss import SparseCategoricalFocalLoss
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import keras
tf.get_logger().setLevel('ERROR')

#from nltk.corpus import stopwords  
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer
#from string import punctuation
#from collections import Counter 
#import wordcloud
#from nltk.corpus import stopwords 
#from textblob import Word 
#import re
#import nltk

In [2]:
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('punkt')

In [3]:
final_data = pd.read_csv('../input/nlpresearch/Data.csv')
final_data = final_data[final_data['maslow'] != 'none']
final_data = final_data[['sentence','maslow']]
final_none = pd.read_csv('../input/nlpresearch/final_none_orig_french_spanish.csv')
frames = [final_data, final_none]
result = pd.concat(frames)
from sklearn.utils import shuffle
result = shuffle(result)
result['maslow'].unique()

In [4]:
result.head()

In [5]:
final_data = result.copy()
final_data[['sentence','maslow']].groupby('maslow').count()

In [6]:
final_data['maslow'].unique()

In [7]:
type2 = final_data.copy()
type2['maslow'] = type2['maslow'].replace(['physiological','love','stability','esteem','spiritual growth'],['basic_needs','basic_needs','basic_needs','advanced_needs','advanced_needs'])
print(type2['maslow'].unique())

In [8]:
final_data[['sentence','maslow']].groupby('maslow').count()

In [9]:
type2[['sentence','maslow']].groupby('maslow').count()

In [10]:
df2oftype2 = type2.copy()
lb = LabelEncoder() 
df2oftype2['maslowLabel'] = lb.fit_transform(df2oftype2['maslow'])
print(df2oftype2['maslow'].unique())
print(df2oftype2['maslowLabel'].unique())

In [11]:
df2oftype2.info()

In [12]:
df2oftype2.head()

In [13]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [14]:
# Note the training variable in the Batch Normalization function. This is required because Batch Normalization operates differently during training vs. the application stage– during training the z score is computed using the batch mean and variance, while in inference, it’s computed using a mean and variance estimated from the entire training set.

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(40, activation = 'relu', name = 'layer1')(l)
l = tf.keras.layers.BatchNormalization()(l, training= True)
l = tf.keras.layers.Dense(40, activation = None, name = 'layer2')(l)
#l = tf.keras.layers.BatchNormalization()(l, training= True)
#l = tf.keras.activations\
#           .relu(l)
#l = tf.keras.layers.Dense(40, activation = None, name = 'layer3')(l)
l = tf.keras.layers.Dense(3, activation='softmax', name="output")(l)

In [15]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [16]:
model.summary()

In [17]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.CategoricalCrossentropy(name='categorical_cross_entropy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      'accuracy'
]

def focal_loss(gamma=2., alpha=4.):   

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

model.compile(loss= focal_loss(alpha=0.1, gamma = 2) , optimizer='adam', metrics=METRICS)
x_train, x_test, y_train, y_test = train_test_split(df2oftype2['sentence'],df2oftype2['maslowLabel'],test_size = 0.2, random_state = 0, stratify = df2oftype2['maslowLabel'])
x_train_text = x_train 
x_test_text = x_test
y_train_text = y_train
y_test_text = y_test
print(x_train.shape)
print(x_test.shape)
#print(x_val.shape)

In [18]:
unique, counts = np.unique(y_train, return_counts=True)
result = np.column_stack((unique, counts)) 
print (result)

In [19]:
unique, counts = np.unique(y_test, return_counts=True)
result = np.column_stack((unique, counts)) 
print (result)

In [20]:
y_train = to_categorical(y_train)
y_test =  to_categorical(y_test)

In [21]:
y_train

In [22]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [23]:
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,

)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                                 factor = 0.2,
                                                 patience = 1,
                                                 verbose = 1,
                                                 min_delta = 1e-4,
                                                 min_lr = 1e-7,
                                                 mode = 'max')          # Reduce learning rate when a metric has stopped improving.

# Model weights are saved at the end of every epoch, if it's the best seen
# so far.
history1 = model.fit(x_train, y_train,epochs=20, verbose=1, validation_data = (x_test,y_test),callbacks=[model_checkpoint_callback,callback, reduce_lr])

In [25]:
model.load_weights(checkpoint_filepath)
model.evaluate(x_test, y_test)

In [26]:
print(history1.history.keys())

In [27]:
plt.plot(history1.history['loss'], '-o', label='Train Loss')
plt.plot(history1.history['val_loss'], '-o', label = 'Val Loss')
plt.legend()

In [28]:
plt.plot(history1.history['accuracy'], '-o', label='Train Accuracy')
plt.plot(history1.history['val_accuracy'], '-o', label = 'Val Accuracy')
plt.legend()

In [29]:
plt.plot(history1.history['precision'], '-o', label='Train Precision')
plt.plot(history1.history['recall'], '-o', label = 'Train Recall')
plt.plot(history1.history['val_precision'], '-o', label='Val Precision')
plt.plot(history1.history['val_recall'], '-o', label = 'Val Recall')
plt.legend()

In [30]:
y_pred2 = model.predict(x_test)

In [31]:
y_pred2[2]

In [32]:
review_predictions2 = []
for prediction in y_pred2:
    x = prediction
    review_predictions2.append(np.argmax(x))

review_predictions2 = np.array(review_predictions2)
review_predictions2.reshape(1,-1)
review_predictions2.shape

In [33]:
review_predictions2

In [34]:
k = pd.DataFrame(review_predictions2, columns = ['Label'])
k.groupby('Label')['Label'].count()

In [35]:
text_label = pd.Series(lb.inverse_transform(y_test_text))
integer_label = pd.Series(y_test_text)

In [36]:
integer_label = integer_label.reset_index(drop = True)

In [37]:
def make_confusion_matrix(cf,group_names=None,categories='auto',count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.
    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

In [38]:
df2oftype2.head()
uniqueClassesMaslowLabel2 = list(df2oftype2['maslow'].unique())
uniqueClassesMaslow2 = list(df2oftype2['maslowLabel'].unique())
uniqueClassesMaslowLabel2 = ['advanced_needs','basic_needs','none']
uniqueClassesMaslow2 = [0,1,2]
print(uniqueClassesMaslowLabel2)
print(uniqueClassesMaslow2)
print(classification_report(review_predictions2, y_test_text,target_names = uniqueClassesMaslowLabel2))

In [39]:
import seaborn as sns
r2 = sklearn.metrics.confusion_matrix(review_predictions2, y_test_text)
print(r2)
ax = sns.heatmap(r2, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

In [40]:
make_confusion_matrix(r2, group_names=uniqueClassesMaslow2, categories=uniqueClassesMaslowLabel2, cmap='Blues')

In [41]:
print(uniqueClassesMaslowLabel2)
print(uniqueClassesMaslow2)

In [42]:
x_test_text = x_test_text.reset_index(drop = True)
x_test_text[0:5]

In [43]:
y_test

In [44]:
count = 0
for i in range(0,len(review_predictions2)):
    if review_predictions2[i] != 0 and integer_label[i] == 0 and count != 10:
        print(x_test_text[i])
        count = count + 1